# Custom Loss

단순 선형 회귀를 활용한 Custom Loss를 만들어 봅니다.

## import

In [1]:
import tensorflow as tf
import numpy as np

## data

더미 데이터를 생성합니다.

다음의 데이터는 $y = 2x - 1$ 관계를 가지고 있습니다.

In [2]:
# inputs
xs = np.array([-1.0,  0.0, 1.0, 2.0, 3.0, 4.0], dtype=float)

# labels
ys = np.array([-3.0, -1.0, 1.0, 3.0, 5.0, 7.0], dtype=float)

## train

`mean_squared_error` loss를 활용하여 모델을 train합니다.

In [3]:
model = tf.keras.Sequential([
    tf.keras.layers.Dense(units=1, input_shape=[1])
])

model.compile(optimizer='sgd', loss='mean_squared_error')
model.fit(xs, ys, epochs=500,verbose=0)

print(model.predict([10.0]))

[[18.981628]]


## Custom Loss (1)

Custom Loss를 만듭니다.

1. 함수를 정의합니다.
2. `y_true`와 `y_pred` 변수가 필요합니다.
3. 두 label간의 loss 를 계산하여 return 하는 함수로 만듭니다.

In [4]:
def my_huber_loss(y_true, y_pred):
    # huber loss threshold
    threshold = 1
    # error
    error = y_true - y_pred
    
    # small error flag
    is_small_error = tf.abs(error) <= threshold
    
    # l2 loss 적용
    small_error_loss = tf.square(error) / 2
    
    # l1 loss 적용
    big_error_loss = threshold * (tf.abs(error) - (0.5 * threshold))
    
    # return
    return tf.where(is_small_error, small_error_loss, big_error_loss)

 `model.compile()`에서 loss 매개변수에 custom loss를 적용합니다.

In [5]:
model = tf.keras.Sequential([
    tf.keras.layers.Dense(units=1, input_shape=[1])
])

model.compile(optimizer='sgd', loss=my_huber_loss)
model.fit(xs, ys, epochs=500,verbose=0)

# 결과 확인
print(model.predict([10.0]))

[[18.809072]]


## Custom Loss (2)

위의 `my_huber_loss()`는 우리가 `threshold`에 대한 값을 지정할 수 없다는 단점이 존재합니다.

만약, `threshold` 값을 parameter로 넘겨 주고 싶다면 어떻게 할까요?

### wrapper functions에 대한 이해

In [6]:
def outer_func(multiple):
    def inner_func(a, b):
        result = (a + b) * multiple
        return result
    return inner_func

In [7]:
outer_func(2)

<function __main__.outer_func.<locals>.inner_func(a, b)>

In [8]:
outer_func(2)(1, 3)

8

### wrapper function으로 custom loss 생성

In [11]:
def my_huber_loss(threshold=1.0):
    def inner_loss(y_true, y_pred):
        # huber loss threshold
        # threshold = 1
        # error
        error = y_true - y_pred

        # small error flag
        is_small_error = tf.abs(error) <= threshold

        # l2 loss 적용
        small_error_loss = tf.square(error) / 2

        # l1 loss 적용
        big_error_loss = threshold * (tf.abs(error) - (0.5 * threshold))

        # return
        return tf.where(is_small_error, small_error_loss, big_error_loss)
    return inner_loss

In [23]:
model = tf.keras.Sequential([
    tf.keras.layers.Dense(units=1, input_shape=[1])
])

model.compile(optimizer='sgd', loss=my_huber_loss(threshold=1.2))
model.fit(xs, ys, epochs=500,verbose=0)

# 결과 확인
print(model.predict([10.0]))

[[18.636951]]
